# **Youtube video summarizer**

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
srt = YouTubeTranscriptApi.get_transcript("KTonvXhsxpc")

: 

In [ ]:
# writing to a file 
with open("subtitles.txt", "w") as f:
    for i in srt:
        f.write("{}\n".format(i))

In [ ]:
# storing file content in a string
with open('subtitles.txt') as f:
    input_text = f.read()
input_text

In [ ]:
# text splitting 
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

documents = text_splitter.create_documents([input_text])
documents


In [ ]:
# storing vectors to vector store
from langchain.embeddings import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS

embedding = OllamaEmbeddings(model="mxbai-embed-large:latest") 
db = FAISS.from_documents(documents, embedding)



In [ ]:
# vector store as retriever
retriever = db.as_retriever()

In [ ]:
# getting the main content from vector store
docs = retriever.get_relevant_documents('What is the main idea of the video?')
docs

In [ ]:
# initializing the model for summarization 
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
)

In [ ]:
# prompt template
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    "Summarize the following text:\n\n{text}\n\nSummary: and dont include unnecessary text start directlly by generating the summary"
)

In [ ]:
# creating RAG chain
from langchain.chains import LLMChain

summary_chain = LLMChain(llm=llm, prompt=prompt)
retrieved_text = "\n\n".join([doc.page_content for doc in docs])  # Combine small docs
summary = summary_chain.run({"text": retrieved_text})


In [ ]:
print(summary)